In [1]:
%load_ext autoreload
%autoreload 2

<div style="font-family: Vazirmatn; font-size:30px; direction: rtl; display: flex; align-items:center; justify-content:center; align-text:center; background-color: #d6336c; padding: 20px 40px;border: 1px solid black; border-radius: 5px">
مدلسازی و حل مسئله
</div>

In [2]:
# ---------------------------------------------------------------------------- #
#                              Import the modules                              #
# ---------------------------------------------------------------------------- #
from pyomo.environ import *

# ---------------------------------------------------------------------------- #
#                             Initialize the MODEL                             #
# ---------------------------------------------------------------------------- #

model = ConcreteModel()

# ---------------------------------------------------------------------------- #
#                               Define the PARAMS                              #
# ---------------------------------------------------------------------------- #
products = ['A', 'B', 'C', 'D', 'E']
D = {'A': 4000, 'B': 15000, 'C': 2000, 'D': 8000, 'E': 1000}
A = {'A': 100, 'B': 120, 'C': 250, 'D': 250, 'E': 120}
t = {'A': 0.2, 'B': 0.01, 'C': 0.1, 'D': 0.05, 'E': 0.2} 
P = {j: (1 / t[j]) * 2000 for j in products}
D_P = {j: D[j]/P[j] for j in products}
c = {'A': 5, 'B': 10, 'C': 20, 'D': 10, 'E': 5}  
i = 0.2

# ---------------------------------------------------------------------------- #
#                             Define the VARIABLES                             #
# ---------------------------------------------------------------------------- #
model.T = Var(domain=NonNegativeReals) 

# ---------------------------------------------------------------------------- #
#                         Define the OBJECTIVE FUNCTION                        #
# ---------------------------------------------------------------------------- #
model.objective = Objective(
    expr=sum((A[j] * (1/model.T)) + (c[j] * D[j]) + (i * c[j] * model.T * D[j] * 0.5) * (1-D_P[j]) for j in products),
    sense=minimize
)

# ---------------------------------------------------------------------------- #
#                                  Constraints                                 #
# ---------------------------------------------------------------------------- #

# ----------------- 1. Is it possible to satisfy the demand in 1 year ---------------- #
feasibility_constraint = sum((D_P[j]) for j in products) <= 1

# ---------------------------------------------------------------------------- #
#                                Solve the model                               #
# ---------------------------------------------------------------------------- #
print("Is it feasible?", feasibility_constraint)
solver = SolverFactory('ipopt')
solver.solve(model)

print(f'objective function : {value(model.objective)}')

print(value(model.T))
X = {j: value(model.T) * D[j] for j in products}
print(X)

Is it feasible? True


objective function : 304260.8854868204
0.1814081388212119
{'A': 725.6325552848476, 'B': 2721.1220823181784, 'C': 362.8162776424238, 'D': 1451.2651105696953, 'E': 181.4081388212119}
